# বাংলা  sequence generator

এই প্রজেক্টে, বাংলায় ক্যারেক্টার-বাই-ক্যারেক্টার sequence generator তৈরি করা হয়েছে। 
প্রজেক্টে, ডেটাসেট হিসেবে  প্রয়াত লেখক হুমায়ুন আহমেদ এর বিখ্যাত বই "হিমু" এর corpus ইউজ করা হয়েছে। "হিমু" বইয়ে প্রায় 115440 ক্যারেক্টার আছে যা ট্রেইন করার জন্য যথেষ্ট।  



In [1]:
import re
import numpy as np
### necessary functions from the keras library
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import keras
import random

Using TensorFlow backend.
/Applications/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


##   বাংলা টেক্সট ডেটাসেট এর প্রি-প্রসেস

প্রথমে, আমরা কিছু প্রি-প্রসেস মেথড ইউজ করবো যা আমাদেরকে বিশাল টেক্সট corpus ট্রেইন করতে সুবিধা দিবে।  

In [2]:
# হিমু বইয়ের টেক্সট গুলো read করবো। 
bangla_text = open('datasets/himu.txt').read()
print('আমাদের অরিজিনাল টেক্সট এ ' + str(len(bangla_text)) + ' বর্ণ আছে')

আমাদের অরিজিনাল টেক্সট এ 115440 বর্ণ আছে


আমরা যেহেতু স্বয়ংক্রিয়ভাবে বর্ণ সমন্বয়ে বাংলা শব্দ তৈরি করবো সেহেতু আমাদেরকে শুধুমাত্র বাংলা বর্ন ছাড়া বাকি ক্যারেক্টারগুলো রিমুভ করে দিবো। 

আমরা কি করতে যাচ্ছি তার একটু ধারণা পাওয়ার জন্য নিচে প্রথম ২০০০ টা বর্ণ প্রিন্ট করছি। 

In [3]:
bangla_text[:2000]

'\nপ্রসঙ্গ হিমু\n\n\n\n\tহিমু আমার প্রিয় চরিত্রের একটি। যখন হিমুকে নিয়ে কিছু লিখি-- নিজেকে হিমু মনে হয়, একধরনের ঘোর অনুভব করি। এই ব্যাপারটা অন্য কোন লেখার সময় তেমন ঘটে না। হিমুকে নিয়ে আমার প্রথম লেখা ময়ুরাক্ষি। ময়ুরাক্ষি লেখার সময় ব্যাপারটা প্রথম লক্ষ করি। দ্বিতীয়বারে লিখলাম রজার ওপাশে। তখনো একই ব্যাপার। কেন এরকম হয়? মানুষ হিসেবে আমি যুক্তিবাদী। হিমু যুক্তিহীন, রহস্যময় জগৎ একজন যুক্তিবাদীকে কেন আকর্ষণ করবে? আমার জানা নেই। যদি কখনও জানতে পারি-- পাঠকদের জানাব।\n\n\n\n\tহুমায়ূন আহমেদ\n\n\tএলিফেন্ট রোড\n\n\n\n\n\nকি নাম বললেন আপনার, হিমু?’\n\n\t‘জ্বি, হিমু।’\n\n\t‘হিম থেকে হিমু?’\n\n\t‘জ্বি-না, হিমালয় থেকে হিমু। আমার ভাল নাম হিমালয়।\n\n\t‘ঠাট্টা করছেন?’\n\n\t‘না, ঠাট্টা করছি না।’\n\n\tআমি পাঞ্জাবির পকেট থেকে ম্যাট্রিক সার্টিফিকেট বের করে এগিয়ে দিলাম।\n\n\tহাসিমুখে বললাম, সার্টিফিকেটে লেখা আছে। দেখুন।\n\n\tএষা হতভম্ব হয়ে বলল, আপনি কি সার্টিফিকেট পকেটে নিয়ে ঘুরে বেড়ান?\n\n\t‘জ্বি, সার্টিফিকেটটা পকেটেই রাখি।হিমালয় নাম বললে অনেকেই বিশ্বাস করে না, তখন সার্টিফিকেট দেখাই। ওরা ত

এখন আমরা টেক্সট এর মধ্য থেকে নতুন লাইন এবং ট্যাব এর Tag গুলো খুঁজে বের করে সেগুলোকে স্পেস দিয়ে রিপ্লেস করবো। 

In [4]:
def remove_tabs(text):
    
    text = text.replace('\n',' ') 
    text = text.replace('\t',' ')
    text = text.replace('\r',' ')
    text = text.replace('\u200d',' ')
    
    text = re.sub(' +',' ',text)
    
    return text

bangla_text = remove_tabs(bangla_text)

পুনরায় আমরা প্রথম ২০০০ লাইন প্রিন্ট করলে দেখতে পারবো যে, টেক্সট এ নতুন লাইন এবং ট্যাব এর Tag গুলো স্পেস দ্বারা রিপ্লেস হয়ে গিয়েছে। 

In [5]:
bangla_text[:2000]

' প্রসঙ্গ হিমু হিমু আমার প্রিয় চরিত্রের একটি। যখন হিমুকে নিয়ে কিছু লিখি-- নিজেকে হিমু মনে হয়, একধরনের ঘোর অনুভব করি। এই ব্যাপারটা অন্য কোন লেখার সময় তেমন ঘটে না। হিমুকে নিয়ে আমার প্রথম লেখা ময়ুরাক্ষি। ময়ুরাক্ষি লেখার সময় ব্যাপারটা প্রথম লক্ষ করি। দ্বিতীয়বারে লিখলাম রজার ওপাশে। তখনো একই ব্যাপার। কেন এরকম হয়? মানুষ হিসেবে আমি যুক্তিবাদী। হিমু যুক্তিহীন, রহস্যময় জগৎ একজন যুক্তিবাদীকে কেন আকর্ষণ করবে? আমার জানা নেই। যদি কখনও জানতে পারি-- পাঠকদের জানাব। হুমায়ূন আহমেদ এলিফেন্ট রোড কি নাম বললেন আপনার, হিমু?’ ‘জ্বি, হিমু।’ ‘হিম থেকে হিমু?’ ‘জ্বি-না, হিমালয় থেকে হিমু। আমার ভাল নাম হিমালয়। ‘ঠাট্টা করছেন?’ ‘না, ঠাট্টা করছি না।’ আমি পাঞ্জাবির পকেট থেকে ম্যাট্রিক সার্টিফিকেট বের করে এগিয়ে দিলাম। হাসিমুখে বললাম, সার্টিফিকেটে লেখা আছে। দেখুন। এষা হতভম্ব হয়ে বলল, আপনি কি সার্টিফিকেট পকেটে নিয়ে ঘুরে বেড়ান? ‘জ্বি, সার্টিফিকেটটা পকেটেই রাখি।হিমালয় নাম বললে অনেকেই বিশ্বাস করে না, তখন সার্টিফিকেট দেখাই। ওরা তখন বড় ধরণের ঝাঁকি খায়।’ আমি উঠে দাড়ালাম।এষা বলল, আপনি কি চলে যাচ্ছেন? ‘হুঁ।’ 

আমাদের টেক্সট এ অনেকগুলো ক্যারেক্টার বা সিম্বল আছে যেগুলো আমাদের কাছে আসবেনা। তাই সেই সকল
স্পেশাল সিম্বল গুলো খুঁজে বের করে সেগুলোকে স্পেস দ্বারা রিপ্লেস করে দিবো। 

In [6]:
def remove_symbols(txt):
    
    chars = ['$', '!', '@', '?', '%', '/', '*', '-', '&', '(', ')', '"', "'", ',', ';', ':',
             '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0','‘','।','’','—', 
              '১', '২', '৩', '\u200c', '–', '“', '”', '…',
              'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's',
             't', 'u', 'w', 'y', 'z', '।', ',', '?','I', 'K', 'R', 'T', 'Y'] 
    for c in chars:
        if c in txt:
            txt = txt.replace(c, " ")

    # যদি অতিরিক্ত স্পেস থাকে তাহলে একটা স্পেস এ রিপ্লেস করবে। 
    txt = txt.replace('  ',' ')
    
    return txt
    
bangla_text = remove_symbols(bangla_text)

পুনরায় আমরা প্রথম ২০০০ লাইন প্রিন্ট করলে দেখে যাচাই করবো যে আগের থেকে দেখতে সুন্দর দেখাচ্ছে কিনা। 

In [7]:
bangla_text[:2000]

' প্রসঙ্গ হিমু হিমু আমার প্রিয় চরিত্রের একটি যখন হিমুকে নিয়ে কিছু লিখি  নিজেকে হিমু মনে হয় একধরনের ঘোর অনুভব করি এই ব্যাপারটা অন্য কোন লেখার সময় তেমন ঘটে না হিমুকে নিয়ে আমার প্রথম লেখা ময়ুরাক্ষি ময়ুরাক্ষি লেখার সময় ব্যাপারটা প্রথম লক্ষ করি দ্বিতীয়বারে লিখলাম রজার ওপাশে তখনো একই ব্যাপার কেন এরকম হয় মানুষ হিসেবে আমি যুক্তিবাদী হিমু যুক্তিহীন রহস্যময় জগৎ একজন যুক্তিবাদীকে কেন আকর্ষণ করবে আমার জানা নেই যদি কখনও জানতে পারি  পাঠকদের জানাব হুমায়ূন আহমেদ এলিফেন্ট রোড কি নাম বললেন আপনার হিমু  জ্বি হিমু  হিম থেকে হিমু  জ্বি না হিমালয় থেকে হিমু আমার ভাল নাম হিমালয়  ঠাট্টা করছেন  না ঠাট্টা করছি না  আমি পাঞ্জাবির পকেট থেকে ম্যাট্রিক সার্টিফিকেট বের করে এগিয়ে দিলাম হাসিমুখে বললাম সার্টিফিকেটে লেখা আছে দেখুন এষা হতভম্ব হয়ে বলল আপনি কি সার্টিফিকেট পকেটে নিয়ে ঘুরে বেড়ান  জ্বি সার্টিফিকেটটা পকেটেই রাখি হিমালয় নাম বললে অনেকেই বিশ্বাস করে না তখন সার্টিফিকেট দেখাই ওরা তখন বড় ধরণের ঝাঁকি খায়  আমি উঠে দাড়ালাম এষা বলল আপনি কি চলে যাচ্ছেন  হুঁ  এখন যাবেন না একটু বসুন  আমার যেহেতু কখনোই কো

সো, এখন আমাদের সুন্দর একটি বাংলা ক্যারেক্টার sequence আছে। এখন টোটাল ক্যারেক্টার সংখ্যা এবং টোটাল ইউনিক ক্যারেক্টার সংখ্যা প্রিন্ট করবো। 

In [10]:
# টোটাল ইউনিক ক্যারেক্টার গণনা 
chars = sorted(list(set(bangla_text)))

print ("এই corpus এ আছে সর্বমোট " +  str(len(bangla_text)) + " টি ক্যারেক্টার ")
print ("এই corpus এ আছে সর্বমোট " +  str(len(chars)) + " টি ইউনিক ক্যারেক্টার")

এই corpus এ আছে সর্বমোট 107843 টি ক্যারেক্টার 
এই corpus এ আছে সর্বমোট 61 টি ইউনিক ক্যারেক্টার


### ইনপুট/আউটপুট যুগল তৈরি

In [11]:
def window_transform_text(text,window_size,step_size):
    # containers for input/output pairs
    inputs = [text[i:i+window_size] for i in range(0, len(text)-window_size, step_size)]
    outputs = [text[i] for i in range(window_size, len(text), step_size)]    
    return inputs,outputs

উপরের ফাংশন এর সাহায্যে ইনপুট/আউটপুট যুগল তৈরি করতে পারি। যেমনঃ- 


In [12]:
# run your text window-ing function 
window_size = 100
step_size = 5
inputs, outputs = window_transform_text(bangla_text,window_size,step_size)

নিচে কিছু ইনপুট/আউটপুট যুগল প্রিন্ট করলাম। 

In [13]:
# print out a few of the input/output pairs to verify that we've made the right kind of stuff to learn from
print('input = ' + inputs[1])
print('output = ' + outputs[1])
print('--------------')
print('input = ' + inputs[302])
print('output = ' + outputs[302])

input = ঙ্গ হিমু হিমু আমার প্রিয় চরিত্রের একটি যখন হিমুকে নিয়ে কিছু লিখি  নিজেকে হিমু মনে হয় একধরনের ঘোর অনু
output = ভ
--------------
input = িতে না পেরে পরিবারটা একটু অস্বস্তির মধ্যে পড়েছে ঘরে বোধহয় চা পাতা নেই চা পাতা থাকলে এতক্ষণে চা চলে আ
output = স


## টেক্সট তৈরি 


ক্যারেক্টার-বাই-ক্যারেক্টার টেক্সট তৈরি হচ্ছে একটি ক্লাসিফিকেশন প্রবলেম কারণ প্রতিটি আউটপুট হচ্ছে একটি সিঙ্গেল ক্যারেক্টার। তাই প্রেডিকশনের ক্লাসের সংখ্যা হচ্ছে ইউনিক ক্যারেক্টারের সমান।

In [14]:
# print out the number of unique characters in the dataset
chars = sorted(list(set(bangla_text)))
print ("this corpus has " +  str(len(chars)) + " unique characters")
print ('and these characters are ')
print (chars)

this corpus has 61 unique characters
and these characters are 
[' ', '.', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'এ', 'ঐ', 'ও', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়']


## 2.5  One-hot encoding characters

যেহেতু আমাদের ইউনপুট/আউটপুট যুগল হচ্ছে ক্যারেক্টার সেহেতু বাংলা টেক্সট এর প্রতিটি ক্যারেক্টার সমূহকে তার নিউমেরিক ভ্যালুতে নিতে হবে। এই কাজটি আমরা One-hot ইউকোডিং এর মাধ্যমে করতে পারি। 

$$অ\longleftarrow\left[\begin{array}{c}
1\\
0\\
0\\
\vdots\\
0\\
0
\end{array}\right]\,\,\,\,\,\,\,আ\longleftarrow\left[\begin{array}{c}
0\\
1\\
0\\
\vdots\\
0\\
0
\end{array}\right]\,\,\,\,\,ই\longleftarrow\left[\begin{array}{c}
0\\
0\\
1\\
\vdots\\
0\\
0 
\end{array}\right]\cdots$$



In [16]:
# this dictionary is a function mapping each unique character to a unique integer
chars_to_indices = dict((c, i) for i, c in enumerate(chars))  # map each unique character to unique integer

# this dictionary is a function mapping each unique integer back to a unique character
indices_to_chars = dict((i, c) for i, c in enumerate(chars))  # map each unique integer back to unique character

এখন আমরা আমাদের  ইউনপুট/আউটপুট যুগল তৈরি করে সেগুলোকে One-hot ভেক্টর এ নিয়েছি। 

In [17]:
# transform character-based input/output into equivalent numerical versions
def encode_io_pairs(text,window_size,step_size):
    # number of unique chars
    chars = sorted(list(set(text)))
    num_chars = len(chars)
    
    # cut up text into character input/output pairs
    inputs, outputs = window_transform_text(text,window_size,step_size)
    
    # create empty vessels for one-hot encoded input/output
    X = np.zeros((len(inputs), window_size, num_chars), dtype=np.bool)
    y = np.zeros((len(inputs), num_chars), dtype=np.bool)
    
    # loop over inputs/outputs and tranform and store in X/y
    for i, sentence in enumerate(inputs):
        for t, char in enumerate(sentence):
            X[i, t, chars_to_indices[char]] = 1
        y[i, chars_to_indices[outputs[i]]] = 1
        
    return X,y

এই ফাংশনটি raw ক্যারেক্টার নিবে এবং তাদের নিউমেরিক্যাল ভ্যালুতে রুপান্তরিত করবে। 

In [19]:

window_size = 100
step_size = 5
X,y = encode_io_pairs(bangla_text,window_size,step_size)

<a id='TODO_5'></a>

##  RNN মডেল তৈরি

আমাদের RNN মডেল এ একটি সিঙ্গেল LSTM হিডেন লেয়ার থাকবে। 

- layer 1 হচ্ছে ২০০ হিডেন ইউনিট এর একটি LSTM মডিউল  --> নোট input_shape = (window_size,len(chars)) যেখানে, len(chars) = number of unique characters
- layer 2 হচ্ছে একটি  linear module, fully connected, with len(chars) hidden units --> যেখানে,  len(chars) = number of unique characters
- layer 3 হচ্ছে একটি softmax activation
- এই মডিউলে **categorical_crossentropy** loss ব্যবহার করেছি।  

In [20]:

def create_rrn_model():

    model = Sequential()
    model.add(LSTM(200, input_shape=(window_size, len(chars))))
    model.add(Dense(len(chars)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

আমাদের মডেল আর্কিটেকচার হচ্ছেঃ- 

In [24]:
model = create_rrn_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 200)               209600    
_________________________________________________________________
dense_3 (Dense)              (None, 61)                12261     
_________________________________________________________________
activation_3 (Activation)    (None, 61)                0         
Total params: 221,861
Trainable params: 221,861
Non-trainable params: 0
_________________________________________________________________


## ট্রেইনিং

প্রথমে আমাদের ডেটাসেট কে ট্রেইনিং এবং টেস্টিং এ বিভক্ত করি। 

In [25]:
# a small subset of our input/output pairs
Xtrain = X[:100000,:,:]
ytrain = y[:100000,:]


In [26]:
def training(model, Xtrain, ytrain, batch_size=500, epochs=50):
    
    model.fit(Xtrain, ytrain, batch_size=500, epochs=50,verbose = 1)
    
    # মডেল সেভ
    model.save('result/test.h5')

In [28]:

training(model, Xtrain, ytrain)

Epoch 1/50
21549/21549 [==============================] - 192s - loss: 3.3573   
Epoch 2/50
21549/21549 [==============================] - 196s - loss: 3.1320   
Epoch 3/50
21549/21549 [==============================] - 191s - loss: 3.0584   
Epoch 4/50
21549/21549 [==============================] - 190s - loss: 2.8953   
Epoch 5/50
21549/21549 [==============================] - 190s - loss: 2.7341   
Epoch 6/50
21549/21549 [==============================] - 190s - loss: 2.6167   
Epoch 7/50
21549/21549 [==============================] - 186s - loss: 2.5356   
Epoch 8/50
21549/21549 [==============================] - 187s - loss: 2.4739   
Epoch 9/50
21549/21549 [==============================] - 186s - loss: 2.4226   
Epoch 10/50
21549/21549 [==============================] - 187s - loss: 2.3804   
Epoch 11/50
21549/21549 [==============================] - 186s - loss: 2.3447   
Epoch 12/50
21549/21549 [==============================] - 187s - loss: 2.3117   
Epoch 13/50
21549/21549 [

How do we make a given number of predictions (characters) based on this fitted model?   

First we predict the next character after following any chunk of characters in the text of length equal to our chosen window size.  Then we remove the first character in our input sequence and tack our prediction onto the end.  This gives us a slightly changed sequence of inputs that still has length equal to the size of our window.  We then feed in this updated input sequence into the model to predict the another character.  Together then we have two predicted characters following our original input sequence.  Repeating this process N times gives us N predicted characters.

In [29]:
# function that uses trained model to predict a desired number of future characters
def predict_next_chars(model,input_chars,num_to_predict):     
    # create output
    predicted_chars = ''
    for i in range(num_to_predict):
        # convert this round's predicted characters to numerical input    
        x_test = np.zeros((1, window_size, 61))
        #print(x_test.shape)
        #x_test.reshape(1, 100, 28)
        for t, char in enumerate(input_chars):
            
            x_test[0, t, chars_to_indices[char]] = 1.

        # make this round's prediction
        test_predict = model.predict(x_test,verbose = 0)[0]

        # translate numerical prediction back to characters
        r = np.argmax(test_predict)                           # predict class of each test input
        d = indices_to_chars[r] 

        # update predicted_chars and input
        predicted_chars+=d
        #print(r)
        input_chars+=d
        input_chars = input_chars[1:]
    return predicted_chars

<a id='TODO_6'></a>

We now try a few subsets of the complete text as input - note the length of each must be exactly equal to the window size.

In [31]:
from keras.models import load_model

# get an appropriately sized chunk of characters from the text
start_inds = [200, 500, 800, 1200]

# load in weights
#model.load_weights('model_weights/best_RNN_small_textdata_weights.hdf5')
new_model = load_model('result/test.h5')
for s in start_inds:
    start_index = s
    input_chars = bangla_text[start_index: start_index + window_size]

    # use the prediction function
    predict_input = predict_next_chars(new_model,input_chars,num_to_predict = 100)

    # print out input characters
    print('------------------')
    input_line = 'input chars = ' + '\n' +  input_chars + ' "' + '\n'
    print(input_line)

    # print out predicted characters
    line = 'predicted chars = ' + '\n' +  predict_input + '"' + '\n'
    print(line)

------------------
input chars = 
্ষি লেখার সময় ব্যাপারটা প্রথম লক্ষ করি দ্বিতীয়বারে লিখলাম রজার ওপাশে তখনো একই ব্যাপার কেন এরকম হয় মা "

predicted chars = 
 বললাম ভাত পারছে আমার সঙ্গে বলল ক্যায়েছেল মারামন্য হবে আমার কেছে আপনি কিন কিনু বললাম আমি বললাম আমি আ"

------------------
input chars = 
িম থেকে হিমু  জ্বি না হিমালয় থেকে হিমু আমার ভাল নাম হিমালয়  ঠাট্টা করছেন  না ঠাট্টা করছি না  আমি পাঞ "

predicted chars = 
্ষু করা  আমি চল করে  বলল মোঝা হচ্ছে আমার মাজে করছে আমি বললাম আমি আমার জন্যে আপনাকে মায়া দিয়ে বললেন আ"

------------------
input chars = 
য় নাম বললে অনেকেই বিশ্বাস করে না তখন সার্টিফিকেট দেখাই ওরা তখন বড় ধরণের ঝাঁকি খায়  আমি উঠে দাড়ালাম এ "

predicted chars = 
ষা বললাম চাষা হয়ে এই মাথা খেল ময় দেখে পড়ি  আমি আছে  আপনা কিছে না তা আমার মন্যে আমার জানার মাথা  আপনা"

------------------
input chars = 
খেলা করছে সে অনেক কিছুই জিজ্ঞেস করতে চাচ্ছে আবার জিজ্ঞেস করতে ভরসা পাচ্ছে না আমি তাদের কাছে নিতান্তই "

predicted chars = 
 ভালবেনি ভালার করছে না তাঁর জানে আমি কিনু কিনে ব

Now we try the same model on the first 100,000 input/output pairs.  

In [ ]:
### A simple way to write output to file
f = open('my_test_output.txt', 'w')              # create an output file to write too
f.write('this is only a test ' + '\n')           # print some output text
x = 2
f.write('the value of x is ' + str(x) + '\n')    # record a variable value
f.close()     

# print out the contents of my_test_output.txt
f = open('my_test_output.txt', 'r')              # create an output file to write too
f.read()

With this recording device enables us to safely perform experiments on larger portions of the text.

First we fit our model to the dataset, then generate text using the trained model in precisely the same generation method applied before on the small dataset.

Finally, we can see the sentences below generated to read like Sherlock Holmes by the neural network

In [32]:
# get an appropriately sized chunk of characters from the text
start_inds = [200,300,400,600,700,1100,1300,1700]

# save output
f = open('text_gen_output/RNN_large_textdata_output.txt', 'w')  # create an output file to write too

# load weights
new_model = load_model('result/test.h5')
for s in start_inds:
    start_index = s
    input_chars = bangla_text[start_index: start_index + window_size]

    # use the prediction function
    predict_input = predict_next_chars(new_model,input_chars,num_to_predict = 100)

    # print out input characters
    line = '-------------------' + '\n'
    print(line)
    f.write(line)

    input_line = 'input chars = ' + '\n' +  input_chars + '"' + '\n'
    print(input_line)
    f.write(input_line)

    # print out predicted characters
    predict_line = 'predicted chars = ' + '\n' +  predict_input + '"' + '\n'
    print(predict_line)
    f.write(predict_line)
f.close()

-------------------

input chars = 
্ষি লেখার সময় ব্যাপারটা প্রথম লক্ষ করি দ্বিতীয়বারে লিখলাম রজার ওপাশে তখনো একই ব্যাপার কেন এরকম হয় মা"

predicted chars = 
 বললাম ভাত পারছে আমার সঙ্গে বলল ক্যায়েছেল মারামন্য হবে আমার কেছে আপনি কিন কিনু বললাম আমি বললাম আমি আ"

-------------------

input chars = 
নুষ হিসেবে আমি যুক্তিবাদী হিমু যুক্তিহীন রহস্যময় জগৎ একজন যুক্তিবাদীকে কেন আকর্ষণ করবে আমার জানা নেই"

predicted chars = 
 আমি  আমি কে আমার জন্যে না আমার কিছু করে পারছেন  আমা কিছু লাগছে  আপনি বল আম কেন হেন আমার  আমার বাসা "

-------------------

input chars = 
 যদি কখনও জানতে পারি  পাঠকদের জানাব হুমায়ূন আহমেদ এলিফেন্ট রোড কি নাম বললেন আপনার হিমু  জ্বি হিমু  হ"

predicted chars = 
িমা সাহেব বললাম চা চান না  আমি চা খাব চান চাখে চলে একটা বললাম না হয় সাই দরখার করছে না তাহিয় এক আমি আ"

-------------------

input chars = 
্জাবির পকেট থেকে ম্যাট্রিক সার্টিফিকেট বের করে এগিয়ে দিলাম হাসিমুখে বললাম সার্টিফিকেটে লেখা আছে দেখু"

predicted chars = 
 হিমু ভাই  বলব কথা পার চিনি কিনি আমার মায়ে করে আম